In [ ]:
from math import sqrt, cos, sin, pi, sqrt, floor, ceil
import rtsvg
rt = rtsvg.RACETrack()

svg_hdr     = '<svg x="0" y="0" width="256" height="256" viewBox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white" />'
svg_ftr     = '</svg>'

#
# radicalCenter() - generated from Google Gemini
#
def radicalCenter(c0, c1, c2):
  """
  Finds the radical center of three circles.

  Args:
    c0: A tuple representing the first circle (x, y, r).
    c1: A tuple representing the second circle (x, y, r).
    c2: A tuple representing the third circle (x, y, r).

  Returns:
    A tuple representing the coordinates of the radical center (x, y), 
    or None if no radical center exists.
  """

  def power_of_point(circle, point):
    """Calculates the power of a point with respect to a circle."""
    x0, y0, r0 = circle
    x, y = point
    distance_squared = (x - x0)**2 + (y - y0)**2
    return distance_squared - r0**2

  def find_radical_axis(c1, c2):
    """Finds the equation of the radical axis of two circles."""
    x1, y1, r1 = c1
    x2, y2, r2 = c2
    # Equation of the radical axis: 
    # 2*(x1 - x2)*x + 2*(y1 - y2)*y + (r2**2 - r1**2 - x2**2 - y2**2 + x1**2 + y1**2) = 0
    A = 2 * (x1 - x2)
    B = 2 * (y1 - y2)
    C = r2**2 - r1**2 - x2**2 - y2**2 + x1**2 + y1**2
    return A, B, C

  # Find the equations of the radical axes
  axis12 = find_radical_axis(c0, c1)
  axis13 = find_radical_axis(c0, c2)
  # Solve the system of linear equations to find the intersection point
  A1, B1, C1 = axis12
  A2, B2, C2 = axis13
  # Check for parallel lines
  if A1 * B2 - A2 * B1 == 0: 
    return None  # No unique solution, circles are coaxial
  # Use Cramer's Rule to solve for x and y
  D = A1 * B2 - A2 * B1
  Dx = C1 * B2 - C2 * B1
  Dy = A1 * C2 - A2 * C1
  x = Dx / D
  y = Dy / D
  return x, y

def circleIntersections(c0, c1):
  """
  Finds the intersection points of two overlapping circles.

  Args:
    c0: A tuple representing the first circle (x0, y0, r0).
    c1: A tuple representing the second circle (x1, y1, r1).

  Returns:
    A list of tuples, each representing an intersection point (x, y). 
    Returns an empty list if no intersections exist.
  """
  (x0, y0, r0) = c0
  (x1, y1, r1) = c1
  # Calculate the distance between the centers
  d = sqrt((x1 - x0)**2 + (y1 - y0)**2)
  # Check for non-intersecting, tangent, or coincident circles
  if d > r0 + r1:         return []  # No intersection
  if d < abs(r0 - r1):    return []  # No intersection (one circle entirely within the other)
  if d == 0 and r0 == r1: return []  # Coincident circles
  # Calculate the distance from the center of c0 to the line segment connecting the centers
  a = (r0**2 - r1**2 + d**2) / (2 * d)
  # Calculate the coordinates of the point on the line segment connecting the centers
  x2 = x0 + a * (x1 - x0) / d
  y2 = y0 + a * (y1 - y0) / d
  # Calculate the height of the right triangle
  h = sqrt(r0**2 - a**2)
  # Calculate the coordinates of the intersection points
  x3 = x2 + h * (y1 - y0) / d
  y3 = y2 - h * (x1 - x0) / d
  x4 = x2 - h * (y1 - y0) / d
  y4 = y2 + h * (x1 - x0) / d
  return [(x3, y3), (x4, y4)]

def circleTangents(c, p):
  cx, cy = (c[0] + p[0])/2.0, (c[1] + p[1])/2.0
  r      = sqrt((c[0]-p[0])**2 + (c[1]-p[1])**2)/2.0
  points = circleIntersections(c, (cx, cy, r))
  return [(p,points[0]), (p,points[1])]

def orthogonalCircle(circles):
  xy_radical_center = radicalCenter(circles[0], circles[1], circles[2])
  _segments_        = circleTangents(circles[0], xy_radical_center)
  l                 = rt.segmentLength(_segments_[0])
  return (xy_radical_center[0], xy_radical_center[1], l)

def outerCircleTangents(c0, c1):
  if c0[2] > c1[2]: c0, c1 = c1, c0
  c_inner    = (c1[0], c1[1], c1[2]-c0[2])
  _segments_ = circleTangents(c_inner, c0)
  _finals_   = []
  for _segment_ in _segments_:
    _segment_[0] # this should be the center of c0
    _segment_[1] # this should be the delta out from the center of c1
    _uv_ = rt.unitVector(((c1[0], c1[1]), (_segment_[1][0], _segment_[1][1])))
    _finals_.append(((c0[0]+_uv_[0]*c0[2], c0[1]+_uv_[1]*c0[2]),(c1[0]+_uv_[0]*c1[2], c1[1]+_uv_[1]*c1[2])))
  return _finals_

def innerCircleTangents(c0, c1):
  if c0[2] > c1[2]: c0, c1 = c1, c0
  c_inner    = (c1[0], c1[1], c1[2]+c0[2]) # sign difference
  _segments_ = circleTangents(c_inner, c0)
  _finals_   = []
  for _segment_ in _segments_:
    _segment_[0] # this should be the center of c0
    _segment_[1] # this should be the delta out from the center of c1
    _uv_ = rt.unitVector(((c1[0], c1[1]), (_segment_[1][0], _segment_[1][1])))
    _finals_.append(((c0[0]-_uv_[0]*c0[2], c0[1]-_uv_[1]*c0[2]),(c1[0]+_uv_[0]*c1[2], c1[1]+_uv_[1]*c1[2]))) # also sign difference
  return _finals_

def svgX(xy, l=1.0, color='#000000', stroke_width=0.5):
  return f'<line x1="{xy[0]-l}" y1="{xy[1]-l}" x2="{xy[0]+l}" y2="{xy[1]+l}" stroke="{color}" stroke-width="{stroke_width}" />' + \
         f'<line x1="{xy[0]+l}" y1="{xy[1]-l}" x2="{xy[0]-l}" y2="{xy[1]+l}" stroke="{color}" stroke-width="{stroke_width}" />'


In [ ]:
#
# Version of the algorithm described here:
# https://www.cut-the-knot.org/Curriculum/Geometry/GeoGebra/CCC_Gergonne.shtml#solution
#
circles = [(45,45,8),(70,70,5),(40,70,3)]
svg_circles = []
for i in range(len(circles)): svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_orthogonal_circle = []
# Step 1: Find the orthogonal circle
circle_orthogonal = orthogonalCircle(circles)
# Step 2 & 3: Find the axis of similtude
_segments_ = outerCircleTangents(circles[0], circles[1])
R = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[0], circles[2])
Q = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[2], circles[1])
P = rt.intersectionPoint(_segments_[0], _segments_[1])
# Step 4: Draw the tangent lines
_segments_ = circleTangents(circles[2], R)
S1,T1      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[1], Q)
S2,T2      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[0], P)
S3,T3      = _segments_[0][1], _segments_[1][1]
# Step 5: draw the inner and outer circles
_inner_ = rt.threePointCircle(S1,S2,S3)
_outer_ = rt.threePointCircle(T1,T2,T3)
svg_orthogonal_circle.append(f'<circle cx="{_inner_[0]}" cy="{_inner_[1]}" r="{_inner_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(f'<circle cx="{_outer_[0]}" cy="{_outer_[1]}" r="{_outer_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(svgX(S1,color='green')+svgX(S2,color='green')+svgX(S3,color='green'))
svg_orthogonal_circle.append(svgX(T1)+svgX(T2)+svgX(T3))
rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_orthogonal_circle) + svg_ftr])

In [ ]:
import random
def makeInscribableCircleExample(_xyr_=(60.0, 60.0, 10.0), r_min=10.0, r_max=30.0):
    overlaps = True
    while overlaps:
        overlaps = False
        _circles_ = []
        for i in range(3):
            a, r = random.random()*2*pi, r_min + random.random() * (r_max - r_min)
            _circle_ = (_xyr_[0]+(_xyr_[2] + r)*cos(a), _xyr_[1]+(r + _xyr_[2])*sin(a), r)
            _circles_.append(_circle_)
        for i in range(3):
            for j in range(i+1, 3):
                if rt.segmentLength((_circles_[i][0:2], _circles_[j][0:2])) < _circles_[i][2] + _circles_[j][2]:
                    overlaps = True
    return _xyr_, _circles_
_inscribed_, _others_ = makeInscribableCircleExample()
svg_example = []
for i in range(len(_others_)): 
    svg_example.append(f'<circle cx="{_others_[i][0]}" cy="{_others_[i][1]}" r="{_others_[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.4" />')
svg_example.append(f'<circle cx="{_inscribed_[0]}" cy="{_inscribed_[1]}" r="{_inscribed_[2]}" fill="none" stroke="black" stroke-width="0.4" />')
rt.tile([svg_hdr + ''.join(svg_example) + svg_ftr])

In [ ]:
circles = _others_
svg_orthogonal_circle = []
# Step 1: Find the orthogonal circle
circle_orthogonal = orthogonalCircle(circles)
# Step 2 & 3: Find the axis of similtude
_segments_ = outerCircleTangents(circles[0], circles[1])
R = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[0], circles[2])
Q = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[2], circles[1])
P = rt.intersectionPoint(_segments_[0], _segments_[1])
# Step 4: Draw the tangent lines
_segments_ = circleTangents(circles[2], R)
S1,T1      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[1], Q)
T2,S2      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[0], P)
S3,T3      = _segments_[0][1], _segments_[1][1]
# Step 5: draw the inner and outer circles
_inner_ = rt.threePointCircle(S1,S2,S3)
_outer_ = rt.threePointCircle(T1,T2,T3)
svg_orthogonal_circle.append(f'<circle cx="{_inner_[0]}" cy="{_inner_[1]}" r="{_inner_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(f'<circle cx="{_outer_[0]}" cy="{_outer_[1]}" r="{_outer_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(svgX(S1,color='green')+svgX(S2,color='green')+svgX(S3,color='green'))
svg_orthogonal_circle.append(svgX(T1)+svgX(T2)+svgX(T3))

svg_circles = []
for i in range(len(circles)):
    svg_circles.append(f'<line x1="{_inner_[0]}" y1="{_inner_[1]}" x2="{circles[i][0]}" y2="{circles[i][1]}" stroke="#ff0000" stroke-width="0.2"/>')
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')

rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_orthogonal_circle) + svg_ftr,
         svg_hdr + ''.join(svg_example) + svg_ftr], spacer=10)

In [ ]:
#
# From ChatGPT (doesn't work -- see below)
#
# Write a python script to calculate the inscribed circle in the middle of three non-overlapping circles, c0, c1, and c2.  Each circle is described as a tuple (x, y, r).
#
import math

def calculate_inscribed_circle(c0, c1, c2):
    """
    Calculate the inscribed circle tangent to three given circles.

    Parameters:
        c0, c1, c2: Tuples representing the circles (x, y, r),
                    where x and y are the circle's center coordinates, and r is the radius.

    Returns:
        A tuple (x, y, r) representing the inscribed circle's center and radius.
    """
    def curvature(x, y, r):
        return 1 / r if r > 0 else -1 / abs(r)

    # Extract circle data
    x0, y0, r0 = c0
    x1, y1, r1 = c1
    x2, y2, r2 = c2

    # Compute curvatures (k0, k1, k2)
    k0 = curvature(x0, y0, r0)
    k1 = curvature(x1, y1, r1)
    k2 = curvature(x2, y2, r2)

    # Calculate curvature of the inscribed circle (k3) using Descartes Circle Theorem
    k3 = k0 + k1 + k2 + 2 * math.sqrt(k0 * k1 + k1 * k2 + k2 * k0)

    # Compute weighted center of the inscribed circle
    x3 = (k0 * x0 + k1 * x1 + k2 * x2 + 2 * math.sqrt(k0 * k1 + k1 * k2 + k2 * k0) * (x0 + x1 + x2)) / k3
    y3 = (k0 * y0 + k1 * y1 + k2 * y2 + 2 * math.sqrt(k0 * k1 + k1 * k2 + k2 * k0) * (y0 + y1 + y2)) / k3

    # Compute radius of the inscribed circle (r3)
    r3 = 1 / k3

    return (x3, y3, r3)

inscribed_circle = calculate_inscribed_circle(circles[0], circles[1], circles[2])
print(inscribed_circle)
svg_chatgpt = []
svg_chatgpt.append(f'<circle cx="{inscribed_circle[0]}" cy="{inscribed_circle[1]}" r="{inscribed_circle[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_chatgpt.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_chatgpt.append(rt.svgText('ChatGPT', 5, 123))

rt.tile([svg_hdr + ''.join(svg_chatgpt) + svg_ftr,
         svg_hdr + ''.join(svg_example) + svg_ftr], spacer=10)

In [ ]:
#
# Gemini Flash 2.0 Experimental -- a little better / every result seems to be within the three circles
#
import math

def distance(p1, p2):
    """Calculates the Euclidean distance between two points."""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def inscribed_circle(c0, c1, c2):
    """
    Calculates the inscribed circle tangent to three non-overlapping circles.

    Args:
        c0: Tuple (x, y, r) representing the first circle.
        c1: Tuple (x, y, r) representing the second circle.
        c2: Tuple (x, y, r) representing the third circle.

    Returns:
        A tuple (x, y, r) representing the inscribed circle, or None if it 
        cannot be calculated (e.g., overlapping input circles, degenerate cases).
        Returns None if the input circles are overlapping or otherwise do not form a proper triangle.
    """

    # Check for invalid input (overlapping or near-overlapping circles)
    if (distance((c0[0], c0[1]), (c1[0], c1[1])) <= c0[2] + c1[2] or
        distance((c0[0], c0[1]), (c2[0], c2[1])) <= c0[2] + c2[2] or
        distance((c1[0], c1[1]), (c2[0], c2[1])) <= c1[2] + c2[2]):
        return None

    # Calculate the radii of the 'inner' circles (adjusted for tangency)
    r0 = c0[2]
    r1 = c1[2]
    r2 = c2[2]

    # Calculate the distances between the centers
    a = distance((c1[0], c1[1]), (c2[0], c2[1]))
    b = distance((c0[0], c0[1]), (c2[0], c2[1]))
    c = distance((c0[0], c0[1]), (c1[0], c1[1]))

    # Heron's formula for the area of the triangle formed by the centers
    s = (a + b + c) / 2
    A = math.sqrt(s * (s - a) * (s - b) * (s - c))

    if A == 0: #Degenerate Triangle
        return None

    # Inradius of the triangle formed by the centers
    r = A / s

    # Calculate the coordinates of the incenter (center of the inscribed circle of the triangle formed by the centers)
    x = (a * c0[0] + b * c1[0] + c * c2[0]) / (a + b + c)
    y = (a * c0[1] + b * c1[1] + c * c2[1]) / (a + b + c)

    return (x, y, r)

inscribed = inscribed_circle(circles[0], circles[1], circles[2])

print(inscribed)
svg_gemini = []
svg_gemini.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_gemini.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_gemini.append(rt.svgText('Gemini 2.0 Flash', 5, 123))

rt.tile([svg_hdr + ''.join(svg_gemini) + svg_ftr,
         svg_hdr + ''.join(svg_example) + svg_ftr], spacer=10)

In [ ]:
#
# Gemini 1.5 Flash // returns None... so, doesn't really work correctly...
#
import math

def inscribed_circle(c0, c1, c2):
  """
  Calculates the inscribed circle in the middle of three non-overlapping circles.

  Args:
    c0: Tuple representing the first circle (x, y, r).
    c1: Tuple representing the second circle (x, y, r).
    c2: Tuple representing the third circle (x, y, r).

  Returns:
    Tuple representing the inscribed circle (x, y, r), or None if no solution exists.
  """

  def distance(p1, p2):
    """Calculates the distance between two points."""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

  def intersection(c1, c2):
    """Calculates the intersection points of two circles."""
    x1, y1, r1 = c1
    x2, y2, r2 = c2
    d = distance((x1, y1), (x2, y2))

    if d > r1 + r2:
      return None  # Circles do not intersect

    a = (r1**2 - r2**2 + d**2) / (2 * d)
    h = math.sqrt(r1**2 - a**2)
    x3 = x1 + a * (x2 - x1) / d
    y3 = y1 + a * (y2 - y1) / d
    x4 = x3 + h * (y2 - y1) / d
    y4 = y3 - h * (x2 - x1) / d
    x5 = x3 - h * (y2 - y1) / d
    y5 = y3 + h * (x2 - x1) / d
    return [(x4, y4), (x5, y5)]

  # Find intersection points of the circles
  int12 = intersection(c0, c1)
  int13 = intersection(c0, c2)
  int23 = intersection(c1, c2)

  if not int12 or not int13 or not int23:
    return None  # No intersection points found

  # Calculate the center of the inscribed circle
  x_sum = 0
  y_sum = 0
  for p1, p2 in [(int12[0], int12[1]), (int13[0], int13[1]), (int23[0], int23[1])]:
    x_sum += (p1[0] + p2[0])
    y_sum += (p1[1] + p2[1])
  x_center = x_sum / 6
  y_center = y_sum / 6

  # Calculate the radius of the inscribed circle
  radius = distance((x_center, y_center), int12[0])

  return (x_center, y_center, radius)


inscribed = inscribed_circle(circles[0], circles[1], circles[2])

print(inscribed)
'''svg_gemini15 = []
svg_gemini15.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_gemini15.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')

rt.tile([svg_hdr + ''.join(svg_gemini15) + svg_ftr,
         svg_hdr + ''.join(svg_example)  + svg_ftr], spacer=10)'''

In [ ]:
#
# Claude 3.5 Sonnet // Similar to the Gemini 2.0 version -- circle always in the middle but rarely (if ever) the right size
#
import math
from typing import Tuple

def calculate_inscribed_circle(c0: Tuple[float, float, float], 
                             c1: Tuple[float, float, float], 
                             c2: Tuple[float, float, float]) -> Tuple[float, float, float]:
    """
    Calculate the inscribed circle that touches three non-overlapping circles.
    
    Args:
        c0, c1, c2: Tuples of (x, y, r) representing circle center coordinates and radius
        
    Returns:
        Tuple of (x, y, r) for the inscribed circle
    """
    x0, y0, r0 = c0
    x1, y1, r1 = c1
    x2, y2, r2 = c2
    
    # Calculate the distances between circle centers
    d01 = math.sqrt((x1 - x0)**2 + (y1 - y0)**2)
    d02 = math.sqrt((x2 - x0)**2 + (y2 - y0)**2)
    d12 = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    # Check if circles overlap
    if d01 <= r0 + r1 or d02 <= r0 + r2 or d12 <= r1 + r2:
        raise ValueError("Circles must not overlap")
    
    # Calculate parameters for the quadratic equation
    a0 = 1.0 / r0
    a1 = 1.0 / r1
    a2 = 1.0 / r2
    
    # Calculate circle inversions
    k0 = x0**2 + y0**2 - r0**2
    k1 = x1**2 + y1**2 - r1**2
    k2 = x2**2 + y2**2 - r2**2
    
    # Calculate determinants
    det = (
        2 * (x0*y1 + x1*y2 + x2*y0 - x0*y2 - x1*y0 - x2*y1)
    )
    
    if abs(det) < 1e-10:
        raise ValueError("Circles are collinear or degenerate")
    
    # Calculate the center coordinates of the inscribed circle
    x = (k0*(y1 - y2) + k1*(y2 - y0) + k2*(y0 - y1)) / det
    y = (k0*(x2 - x1) + k1*(x0 - x2) + k2*(x1 - x0)) / det
    
    # Calculate the radius using one of the original circles
    r = abs(math.sqrt((x - x0)**2 + (y - y0)**2) - r0)
    
    return (x, y, r)

inscribed = calculate_inscribed_circle(circles[0], circles[1], circles[2])

print(inscribed)
svg_claude = []
svg_claude.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_claude.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_claude.append(rt.svgText('Claude', 5, 123))
rt.tile([svg_hdr + ''.join(svg_claude) + svg_ftr])

In [ ]:
def inscribed_circle(c0, c1, c2):
    """
    Given three circles c0, c1, c2, each defined as a tuple (x, y, r),
    compute the circle (X, Y, R) that is externally tangent to all three.
    
    The three tangency conditions are:
      (X - x_i)^2 + (Y - y_i)^2 = (r_i + R)^2   for i = 0, 1, 2.
    """
    # Unpack the circles
    x0, y0, r0 = c0
    x1, y1, r1 = c1
    x2, y2, r2 = c2

    # Subtracting the first equation from the second and third yields:
    #  2*(x0 - x1)X + 2*(y0 - y1)Y = (r1^2 - r0^2) + 2R(r1-r0)
    #                             - [(x1^2-x0^2) + (y1^2-y0^2)]
    #
    # Define the coefficients for these two linear equations:
    A  = 2*(x0 - x1)
    B  = 2*(y0 - y1)
    A2 = 2*(x0 - x2)
    B2 = 2*(y0 - y2)

    F1_0 = (r1**2 - r0**2) - ((x1**2 - x0**2) + (y1**2 - y0**2))
    F1_1 = 2*(r1 - r0)
    F2_0 = (r2**2 - r0**2) - ((x2**2 - x0**2) + (y2**2 - y0**2))
    F2_1 = 2*(r2 - r0)

    # Solve the 2x2 system for X and Y in terms of R.
    # The system is:
    #    A*X + B*Y = F1_0 + F1_1*R
    #    A2*X + B2*Y = F2_0 + F2_1*R
    #
    # Its determinant is:
    D = A * B2 - B * A2
    if abs(D) < 1e-9:
        raise ValueError("Degenerate configuration; the circles may be collinear.")

    # Write X and Y in the form: X = X0 + X1*R, Y = Y0 + Y1*R
    X0 = (F1_0 * B2 - B * F2_0) / D
    X1 = (F1_1 * B2 - B * F2_1) / D
    Y0 = (A * F2_0 - A2 * F1_0) / D
    Y1 = (A * F2_1 - A2 * F1_1) / D

    # Now substitute X = X0 + X1*R, Y = Y0 + Y1*R into the first equation:
    #   (X0 + X1*R - x0)^2 + (Y0 + Y1*R - y0)^2 = (r0+R)^2.
    # Let dx0 = X0 - x0 and dy0 = Y0 - y0.
    dx0 = X0 - x0
    dy0 = Y0 - y0

    # Expanding the equation gives a quadratic in R:
    #   (X1^2+Y1^2 - 1)*R^2 + 2*(dx0*X1+dy0*Y1 - r0)*R + (dx0^2+dy0^2 - r0^2) = 0.
    AR = X1**2 + Y1**2 - 1
    BR = 2*(dx0*X1 + dy0*Y1 - r0)
    CR = dx0**2 + dy0**2 - r0**2

    # Solve the quadratic for R.
    if abs(AR) > 1e-9:
        disc = BR**2 - 4*AR*CR
        if disc < 0:
            raise ValueError("No solution: the discriminant is negative.")
        sqrt_disc = math.sqrt(disc)
        # There are two solutions; we select the positive one.
        R1 = (-BR + sqrt_disc) / (2*AR)
        R2 = (-BR - sqrt_disc) / (2*AR)
        sol = [R for R in (R1, R2) if R > 1e-9]
        if not sol:
            raise ValueError("No positive solution for the radius.")
        R_solution = min(sol)  # Choose the smaller positive radius (the inscribed one).
    else:
        # In the (unlikely) case where AR is nearly zero, solve the linear equation.
        if abs(BR) < 1e-9:
            raise ValueError("Degenerate equation for R.")
        R_solution = -CR / BR
        if R_solution <= 0:
            raise ValueError("No positive solution for the radius.")

    # Now compute the center (X, Y) using the expressions above.
    X_solution = X0 + X1 * R_solution
    Y_solution = Y0 + Y1 * R_solution

    return (X_solution, Y_solution, R_solution)

inscribed = inscribed_circle(circles[0], circles[1], circles[2])
svg_o3_v1 = []
svg_o3_v1.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_o3_v1.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_o3_v1.append(rt.svgText('O3 v1', 5, 123))
rt.tile([svg_hdr + ''.join(svg_o3_v1) + svg_ftr])

In [ ]:
def inscribed_circle(c0, c1, c2):
    """
    Given three circles c0, c1, c2, each defined as a tuple (x, y, r),
    compute the circle (X, Y, R) that is externally tangent to all three.
   
    The tangency conditions are:
      (X - x_i)^2 + (Y - y_i)^2 = (r_i + R)^2, for i = 0, 1, 2.
    """
    # Unpack the circles
    x0, y0, r0 = c0
    x1, y1, r1 = c1
    x2, y2, r2 = c2

    # Subtract the equation for c0 from that for c1 and c2:
    #  2*(x0-x1)X + 2*(y0-y1)Y = (r1^2 - r0^2) - [(x1^2-x0^2)+(y1^2-y0^2)] + 2R(r1 - r0)
    #  2*(x0-x2)X + 2*(y0-y2)Y = (r2^2 - r0^2) - [(x2^2-x0^2)+(y2^2-y0^2)] + 2R(r2 - r0)
    A  = 2*(x0 - x1)
    B  = 2*(y0 - y1)
    A2 = 2*(x0 - x2)
    B2 = 2*(y0 - y2)

    F1_0 = (r1**2 - r0**2) - ((x1**2 - x0**2) + (y1**2 - y0**2))
    F1_1 = 2*(r1 - r0)
    F2_0 = (r2**2 - r0**2) - ((x2**2 - x0**2) + (y2**2 - y0**2))
    F2_1 = 2*(r2 - r0)

    # Solve the 2x2 system for X and Y in terms of R:
    #    A*X + B*Y = F1_0 + F1_1*R
    #    A2*X+ B2*Y = F2_0 + F2_1*R
    D = A * B2 - B * A2
    if abs(D) < 1e-9:
        raise ValueError("Degenerate configuration; the circles may be collinear.")

    # Express X and Y in the form: X = X0 + X1*R, Y = Y0 + Y1*R.
    X0 = (F1_0 * B2 - B * F2_0) / D
    X1 = (F1_1 * B2 - B * F2_1) / D
    Y0 = (A * F2_0 - A2 * F1_0) / D
    Y1 = (A * F2_1 - A2 * F1_1) / D

    # Substitute into the first circle's equation:
    #   (X0 + X1*R - x0)^2 + (Y0 + Y1*R - y0)^2 = (r0+R)^2.
    dx0 = X0 - x0
    dy0 = Y0 - y0

    # This expands to a quadratic in R:
    #   (X1^2+Y1^2 - 1)*R^2 + 2*(dx0*X1+dy0*Y1 - r0)*R + (dx0^2+dy0^2 - r0^2) = 0.
    AR = X1**2 + Y1**2 - 1
    BR = 2*(dx0*X1 + dy0*Y1 - r0)
    CR = dx0**2 + dy0**2 - r0**2

    if abs(AR) > 1e-9:
        disc = BR**2 - 4 * AR * CR
        if disc < 0:
            raise ValueError("No solution: the discriminant is negative.")
        sqrt_disc = math.sqrt(disc)
        R1 = (-BR + sqrt_disc) / (2 * AR)
        R2 = (-BR - sqrt_disc) / (2 * AR)
        # Choose the positive solution(s); typically, the inscribed circle is the smaller positive one.
        sol = [R for R in (R1, R2) if R > 1e-9]
        if not sol:
            raise ValueError("No positive solution for the radius.")
        R_solution = min(sol)
    else:
        # If AR is nearly zero, use the linear solution.
        if abs(BR) < 1e-9:
            raise ValueError("Degenerate equation for R.")
        R_solution = -CR / BR
        if R_solution <= 0:
            raise ValueError("No positive solution for the radius.")

    X_solution = X0 + X1 * R_solution
    Y_solution = Y0 + Y1 * R_solution

    return (X_solution, Y_solution, R_solution)

inscribed = inscribed_circle(circles[0], circles[1], circles[2])
svg_o3_v2 = []
svg_o3_v2.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_o3_v2.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_o3_v2.append(rt.svgText('O3 v2', 5, 123))
rt.tile([svg_hdr + ''.join(svg_o3_v2) + svg_ftr])

In [ ]:
def validate_circles(circles):
    # Check that no two circles overlap – i.e. distance between centers
    # is at least the sum of their radii.
    n = len(circles)
    for i in range(n):
        for j in range(i+1, n):
            (x1, y1, r1) = circles[i]
            (x2, y2, r2) = circles[j]
            d = math.hypot(x2-x1, y2-y1)
            if d < (r1 + r2):
                raise ValueError(f"Circles {i} and {j} overlap (d={d} < r1+r2={r1+r2}).")

def solve_incircle(c0, c1, c2):
    # Unpack circle parameters
    x0, y0, r0 = c0
    x1, y1, r1 = c1
    x2, y2, r2 = c2

    # In the three equations:
    # (X - xi)²+(Y - yi)² = (R + ri)², i=0,1,2,
    # subtract the eq. for circle 0 from that for circle 1 and circle 2.
    A1 = x1 - x0
    B1 = y1 - y0
    C1 = (x1**2 - x0**2 + y1**2 - y0**2) - (r0**2 - r1**2)

    A2 = x2 - x0
    B2 = y2 - y0
    C2 = (x2**2 - x0**2 + y2**2 - y0**2) - (r0**2 - r2**2)

    # The two linear equations become:
    #   A1 * X + B1 * Y = C1 - 2*R*(r0 - r1)
    #   A2 * X + B2 * Y = C2 - 2*R*(r0 - r2)
    #
    # Solve the system: note that X and Y will be linear functions in R.
    det = A1 * B2 - A2 * B1
    if abs(det) < 1e-9:
        raise ValueError("The derived linear equations are degenerate (determinant ~ 0).")

    # Write X = X0 + X1 * R, Y = Y0 + Y1 * R.
    # To do that, note that if we denote:
    #    T1 = C1 - 2R*(r0-r1)
    #    T2 = C2 - 2R*(r0-r2)
    # then:
    #    X = (T1 * B2 - T2 * B1) / det
    # Which separates as:
    #    X = [(C1*B2 - C2*B1) + 2R*(B1*(r0-r2) - B2*(r0-r1))] / det.
    X0 = (C1 * B2 - C2 * B1) / det
    X1 = (2 * (B1*(r0 - r2) - B2*(r0 - r1))) / det

    # Similarly, for Y:
    #    Y = (A1*T2 - A2*T1) / det
    Y0 = (A1 * C2 - A2 * C1) / det
    Y1 = (2 * (A2*(r0 - r1) - A1*(r0 - r2))) / det

    # Thus, incircle center is:
    #    X = X0 + X1 * R , Y = Y0 + Y1 * R.
    #
    # Now substitute in the equation for circle 0:
    #    (X - x0)² + (Y - y0)² = (R + r0)².
    #
    # Write dx = X0 - x0 and dy = Y0 - y0 so that:
    #    (dx + X1*R)² + (dy + Y1*R)² = (R + r0)².
    dx = X0 - x0
    dy = Y0 - y0

    # Expand:
    #    (dx² + 2*dx*X1*R + X1²*R²) + (dy² + 2*dy*Y1*R + Y1²*R²) = R² + 2*r0*R + r0².
    #
    # Rearranging terms leads to a quadratic in R:
    #    (X1² + Y1² - 1)*R² + 2*(dx*X1 + dy*Y1 - r0)*R + (dx² + dy² - r0²) = 0.
    A_quad = (X1**2 + Y1**2 - 1)
    B_quad = 2 * (dx * X1 + dy * Y1 - r0)
    C_quad = dx**2 + dy**2 - r0**2

    # Solve quadratic. Depending on the numbers, A_quad could be very small.
    if abs(A_quad) < 1e-9:
        # Then linear: B_quad * R + C_quad = 0.
        if abs(B_quad) < 1e-9:
            raise ValueError("Degenerate system: cannot solve for R.")
        R = -C_quad / B_quad
    else:
        disc = B_quad**2 - 4 * A_quad * C_quad
        if disc < 0:
            raise ValueError("No real solution for R (discriminant < 0).")
        sqrt_disc = math.sqrt(disc)
        R1 = (-B_quad + sqrt_disc) / (2 * A_quad)
        R2 = (-B_quad - sqrt_disc) / (2 * A_quad)
        # We choose the solution that is positive. In many Apollonius problems
        # there are two tangency circles. Here we want the incircle (the one
        # that fits ‘inside’ the region), so choose the smaller positive radius.
        candidates = [R for R in (R1, R2) if R > 1e-9]
        if not candidates:
            raise ValueError("No positive solution for R.")
        R = min(candidates)

    # Now compute the incircle center.
    X = X0 + X1 * R
    Y = Y0 + Y1 * R

    return (X, Y, R)

inscribed = solve_incircle(circles[0], circles[1], circles[2])
svg_o3_v3 = []
svg_o3_v3.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_o3_v3.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_o3_v3.append(rt.svgText('O3 v3', 5, 123))
rt.tile([svg_hdr + ''.join(svg_o3_v3) + svg_ftr])

In [ ]:
inscribed = rt.inscribedCircle(circles[0], circles[1], circles[2])
svg_rt = []
svg_rt.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_rt.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_rt.append(rt.svgText('O3 v2 (again)', 5, 123))

rt.table([svg_hdr + ''.join(svg_claude)  + svg_ftr,
          svg_hdr + ''.join(svg_gemini)  + svg_ftr,
          svg_hdr + ''.join(svg_chatgpt) + svg_ftr,
          svg_hdr + ''.join(svg_o3_v1)   + svg_ftr,
          svg_hdr + ''.join(svg_o3_v2)   + svg_ftr,
          svg_hdr + ''.join(svg_o3_v3)   + svg_ftr,
          svg_hdr + ''.join(svg_rt)      + svg_ftr,
          svg_hdr + ''.join(svg_example) + svg_ftr], per_row=4, spacer=10)

In [ ]:
def genCircles(x0=16.0, y0=16.0, x1=112.0, y1=112.0, r_min=4.0, r_max=10.0):
    circles = []
    while len(circles) < 3:
        r = random.uniform(r_min, r_max)
        x = random.uniform(x0, x1)
        y = random.uniform(y0, y1)
        overlaps = False
        for i in range(len(circles)):
            if rt.segmentLength((circles[i][0:2], (x, y))) < r + circles[i][2]: overlaps = True
        if overlaps == False: circles.append((x, y, r))
    return circles
_tiles_ = []
for i in range(6*3):
    _circles_      = genCircles()
    _circle_       = rt.inscribedCircle(_circles_[0], _circles_[1], _circles_[2])
    x0, y0, x1, y1 = _circles_[0][0] - _circles_[0][2], _circles_[0][1] - _circles_[0][2], _circles_[0][0] + _circles_[0][2], _circles_[0][1] + _circles_[0][2]
    _tiles_.append(svg_hdr + ''.join([f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="black" stroke-width="1.0" />',
                                      f'<circle cx="{_circles_[0][0]}" cy="{_circles_[0][1]}" r="{_circles_[0][2]}" fill="none" stroke="{rt.co_mgr.getColor(0)}" stroke-width="1" />',
                                      f'<circle cx="{_circles_[1][0]}" cy="{_circles_[1][1]}" r="{_circles_[1][2]}" fill="none" stroke="{rt.co_mgr.getColor(1)}" stroke-width="1" />',
                                      f'<circle cx="{_circles_[2][0]}" cy="{_circles_[2][1]}" r="{_circles_[2][2]}" fill="none" stroke="{rt.co_mgr.getColor(2)}" stroke-width="1" />']) + svg_ftr)
rt.table(_tiles_, per_row=6, spacer=10)

In [ ]:
rt.tile([svg_hdr + ''.join([f'<circle cx="{_circles_[0][0]}" cy="{_circles_[0][1]}" r="{_circles_[0][2]}" fill="none" stroke="{rt.co_mgr.getColor(0)}" stroke-width="1" />',
                            f'<circle cx="{_circles_[1][0]}" cy="{_circles_[1][1]}" r="{_circles_[1][2]}" fill="none" stroke="{rt.co_mgr.getColor(1)}" stroke-width="1" />',
                            f'<circle cx="{_circles_[2][0]}" cy="{_circles_[2][1]}" r="{_circles_[2][2]}" fill="none" stroke="{rt.co_mgr.getColor(2)}" stroke-width="1" />']) + svg_ftr])